In [ ]:
# default_exp model
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler

# Models
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from fastai2.basics import *
from fastai2.callback.progress import *
from fastai2.text.models.qrnn import *

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,clm_shift=[0,-1]),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,clm_shift=[-1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=1000,clm='current')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.dataloaders(get_hdf_files('test_data/'))

## Linear

In [ ]:
#export
class SeqLinear(nn.Module):
    
    def __init__(self,input_size,output_size,hidden_size=100,hidden_layer=1,act=Mish):
        super().__init__()
        def conv_act(inp,out): return nn.Sequential(nn.Conv1d(inp,out,1),act())
        
        if hidden_layer < 1:
            self.lin = nn.Conv1d(input_size,output_size,1)
        else:
            self.lin = nn.Sequential(conv_act(input_size,hidden_size),
                                     *[conv_act(hidden_size,hidden_size) for _ in range(hidden_layer-1)],
                                    nn.Conv1d(hidden_size,output_size,1))
            
    def forward(self, x):
        out = x.transpose(1,2)
        out = self.lin(out)
        return out.transpose(1,2)

## RNNs

In [ ]:
#export
from fastai2.text.models.awdlstm import *
class RNN(nn.Module):
    "inspired by https://arxiv.org/abs/1708.02182"

    def __init__(self, input_size,hidden_size, num_layers, 
                 hidden_p=0.0, input_p=0.0, weight_p=0.0,
                 rnn_type='gru',ret_full_hidden=False,stateful=False,**kwargs):
        super().__init__()
        store_attr(self, 'ret_full_hidden,num_layers,rnn_type,hidden_size,stateful')
        self.bs = 1
        self.rnns = nn.ModuleList([self._one_rnn(input_size if l == 0 else hidden_size,
                                                 hidden_size,weight_p,rnn_type,**kwargs) for l in range(num_layers)])
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(num_layers)])
        self.reset()

    def forward(self, inp, h_init=None):
        bs = inp.shape[0]
        if bs!=self.bs: self._change_hidden(bs)

        if h_init is None and self.stateful: h_init = self._get_hidden()
                
        output = self.input_dp(inp)
        full_hid,new_hidden = [],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            output, h = rnn(output,h_init[l] if h_init is not None else None)
            if l != self.num_layers - 1: output = hid_dp(output)
            full_hid.append(output)
            new_hidden.append(h)
        
        self.hidden =  to_detach(new_hidden, cpu=False, gather=False)
        if self.rnn_type != 'lstm': self.hidden = [tuple([h]) for h in self.hidden]
            
        return output, output if self.ret_full_hidden else new_hidden

    def _get_hidden(self):
        '''retrieve internal hidden state, check if model device has changed'''
        slf_device = one_param(self).device
        if self.hidden[0][0].device != slf_device:
#             import pdb; pdb.set_trace()
            self.hidden = [tuple([to_device(t,slf_device) for t in h]) 
                            for h in self.hidden]
                
        if self.rnn_type == 'lstm': 
            return self.hidden
        else:
            return [h[0] for h in self.hidden]
    
    def _one_rnn(self, n_in, n_out, weight_p, rnn_type,**kwargs):
        "Return one of the inner rnn"
        if rnn_type == 'gru':
            rnn = nn.GRU(n_in, n_out,1,batch_first=True,**kwargs)
            rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'lstm':
            rnn = nn.LSTM(n_in, n_out,1,batch_first=True,**kwargs)
            rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'qrnn':
            rnn = QRNNLayer(n_in, n_out,batch_first=True,**kwargs)
            rnn.linear = WeightDropout(rnn.linear,weight_p,layer_names='weight')
        else:
            raise Exception
        return rnn
    
    def _change_hidden(self, bs):
        '''change hiddenstate if batchsize has changed'''
        self.hidden = [self._change_one_hidden(l, bs) for l in range(self.num_layers)]
        self.bs = bs
    def _change_one_hidden(self, l, bs):
        if self.bs < bs:
            return tuple(torch.cat([h, h.new_zeros(1, bs-self.bs, self.hidden_size)],
                                   dim=1) for h in self.hidden[l])
        if self.bs > bs: 
            return tuple([h[:,:bs] for h in self.hidden[l]])
        return self.hidden[l]
    
    def reset(self):
        "Reset the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.hidden = [self._one_hidden(l) for l in range(self.num_layers)]
    def _one_hidden(self, l):
        "Return one hidden state"
        hid = one_param(self).new_zeros(1, self.bs, self.hidden_size)
        if self.rnn_type == 'lstm':
            return tuple([hid,hid.clone()])
        else:
            return tuple([hid])


In [ ]:
#export
class SimpleRNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,lrn_init_state=False,**kwargs):
        super().__init__()
        self.rnn = RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,**kwargs)
        self.final = nn.Conv1d(hidden_size,output_size,kernel_size=1)
        
        self.init_p = nn.Parameter(torch.ones(size=(num_layers,hidden_size))) if lrn_init_state else None
        
    def forward(self, x):
        if self.init_p is None:
            out,_ = self.rnn(x)
        else:
            out,_ = self.rnn(x,self.init_p.expand((x.shape[0],-1,-1)).transpose(0,1).contiguous())
#         import pdb; pdb.set_trace()
        out = out.transpose(1,2)
        out = self.final(out)
        out = out.transpose(1,2)
        return out
    def reset(self):
        self.rnn.reset()

In [ ]:
model = SimpleRNN(2,1,3,stateful=True)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,11.407500,7.966294,00:02
1,7.444920,0.251261,00:02
2,4.880624,1.397314,00:02
3,3.522378,0.067033,00:02
4,2.648278,0.181598,00:02


In [ ]:
model = SimpleRNN(2,1,3,rnn_type='lstm')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,12.986645,11.725722,00:02


In [ ]:
model = SimpleRNN(2,1,3,rnn_type='qrnn')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,13.519176,12.999596,00:01


## CNNs

In [ ]:
#export
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super().__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=(kernel_size - 1) * dilation,
            dilation=dilation,
            groups=groups,
            bias=bias)
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super().forward(input)[:,:,:-self.__padding]

In [ ]:
#export
@delegates(CausalConv1d, keep=True)
def CConv1D(input_size,output_size,kernel_size=2,activation = Mish, bn = True, **kwargs):
    conv = CausalConv1d(input_size,output_size,kernel_size,**kwargs)
    act = activation() if activation is not None else None
    bn = nn.BatchNorm1d(input_size) if bn else None
    m = [m for m in [bn,conv,act] if m is not None]
    return nn.Sequential(*m)

In [ ]:
#export
class TCN(nn.Module):
    def __init__(self,input_size,output_size,hl_depth=1,hl_width=10,act = Mish):
        super().__init__()
        
        
        self.hl_depth = hl_depth

        self.conv1 = CConv1D(input_size,hl_width)
        self.conv_layers = nn.ModuleList([CConv1D(hl_width,hl_width,dilation=2**(i+1),
                                                  activation=act if i < hl_depth-2 else None)
                                          for i in range(hl_depth-1)])
        
        self.final = nn.Conv1d(hl_width,output_size,kernel_size=1)
        
    def forward(self, x):
        out = x.transpose(1,2)
        out = self.conv1(out)
        for i in range(self.hl_depth-1):
            out = self.conv_layers[i](out)+out
#         print(out.shape)
#         print(self.final.in_features)
        out = self.final(out)
        out = out.transpose(1,2)
        return out

In [ ]:
model = TCN(2,1,hl_depth=3)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,13.597542,13.299940,00:01


In [ ]:
model

TCN(
  (conv1): Sequential(
    (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): CausalConv1d(2, 10, kernel_size=(2,), stride=(1,), padding=(1,))
    (2): Mish()
  )
  (conv_layers): ModuleList(
    (0): Sequential(
      (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): CausalConv1d(10, 10, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
      (2): Mish()
    )
    (1): Sequential(
      (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): CausalConv1d(10, 10, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
    )
  )
  (final): Conv1d(10, 1, kernel_size=(1,), stride=(1,))
)

## CRNNs

In [ ]:
#export
def CRNN(input_size,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=2,hs_cnn=10,hs_rnn=10,
         hidden_p=0.2, input_p=0.0, weight_p=0.5, rnn_type='qrnn'):
    cnn = TCN(input_size,num_ft,hl_depth=num_cnn_layers,hl_width=hs_cnn)
    rnn = SimpleRNN(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn,
                   hidden_p=hidden_p, input_p=input_p, weight_p=weight_p, rnn_type=rnn_type)
    return nn.Sequential(cnn,rnn)

In [ ]:
model = CRNN(2,1,10)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,15.475187,15.510040,00:01


In [ ]:
model = CRNN(2,1,10,rnn_type='gru')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,13.338457,12.935673,00:02


## Autoregressive Models

In [ ]:
#export
class Normalizer1D(nn.Module):
    _epsilon = 1e-16

    def __init__(self, mean, std):
        super(Normalizer1D, self).__init__()
        self.register_buffer('std', std.clone().detach() + self._epsilon)
        self.register_buffer('mean', mean.clone().detach())

    def normalize(self, x):
        return (x-self.mean)/self.std

    def unnormalize(self, x):
        return x*self.std + self.mean

In [ ]:
#export
class AR_Model(nn.Module):
    def __init__(self,model,ar=True,rf=1,hs=False):
        super().__init__()
        self.model = model
        self.ar = ar
        self.rf = rf
        self.hs = hs
        self.norm = None
        
    def init_normalize(self, batch,axes = [0,1]):
        x = batch[1]
        mean = x.mean(axes, keepdim=True)
        std = x.std(axes, keepdim=True)
        self.norm = Normalizer1D(mean,std)
        
    def forward(self, u,y):
        if self.ar:
            y_e = torch.zeros_like(y)
            hs = None
            for i in range(y_e.shape[1]):
                if i < self.rf:
                    y_in = F.pad(y_e[:, :i], [0,0,self.rf-i, 0])
                    u_in = F.pad(u[:, :i+1], [0,0,self.rf-i-1, 0])
                else:
                    y_in = y_e[:, i-self.rf:i]
                    u_in = u[:, i-self.rf+1:i+1]
                    
                if self.norm is not None: y_in=self.norm.normalize(y_in)

                x = torch.cat((u_in, y_in), 2)
                
                if self.hs:
                    y_next,hs = self.model(x,hs)
                else:
                    y_next = self.model(x)
                y_e[:, i] = y_next[:, -1]
            return y_e
        else:
            y_in = F.pad(y[:,:-1,:],[0,0,1,0])
            
            if self.norm is not None: y_in=self.norm.normalize(y_in)
            
            x = torch.cat([u,y_in],dim=2)
            if self.hs:
                y_e,_ = self.model(x)
            else:
                y_e = self.model(x)
            return y_e

In [ ]:
#export
@delegates(RNN, keep=True)
class AR_RNN(nn.Module):
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,**kwargs):
        super().__init__()
        self.rnn = RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,**kwargs)
        self.final = nn.Conv1d(hidden_size,output_size,kernel_size=1)

    def forward(self, x,init_state=None):
#         out = x.transpose(1,2)
        out,hs = self.rnn(x,init_state)
#         import pdb; pdb.set_trace()
        out = out.transpose(1,2)
        out = self.final(out)
        out = out.transpose(1,2)
        return out,hs

In [ ]:
model = AR_Model(AR_RNN(3,1),ar=True,hs=True)
model.init_normalize(db.one_batch())

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_model.ipynb.
Converted 02_learner.ipynb.
Converted 03_tbptt_dl.ipynb.
Converted 11_ProDiag.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_PBT.ipynb.
Converted index.ipynb.
